In [1]:
import os
import datetime
import random
import csv

import PIL
from PIL import Image
from PIL import ImageEnhance

import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import dropout
from tensorflow.contrib.layers import conv2d
from tensorflow.contrib.layers import variance_scaling_initializer
from tensorflow.contrib.layers import batch_norm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# path to dataset 
DATA_ROOT = r"/run/media/onionhuang/HDD/artificial_intelligence/kaggle/Carvana-Image-Masking-Challenge"
assert(os.path.exists(DATA_ROOT))

TRAIN_IMAGES = os.path.join(DATA_ROOT, "train")
assert(os.path.exists(TRAIN_IMAGES))

TRAIN_MASKS = os.path.join(DATA_ROOT, "train_masks")
assert(os.path.exists(TRAIN_MASKS))

TRAIN_MASKS_CSV = os.path.join(DATA_ROOT, "train_masks.csv")
assert(os.path.exists(TRAIN_MASKS_CSV))

METADATA_CSV = os.path.join(DATA_ROOT, "metadata.csv")
assert(os.path.exists(METADATA_CSV))

TEST_IMAGES = os.path.join(DATA_ROOT, "test")
assert(os.path.exists(TEST_IMAGES))

In [2]:
def get_next_batch(iteration, batch_size, dataset):
    if batch_size * (iteration + 1) > len(dataset):
        return dataset[batch_size*iteration:]
    else:
        return dataset[batch_size*iteration:batch_size*(iteration+1)]

In [3]:
def resize_image_pil(
    image, 
    size, 
    random_enhance=True,
    pad=True, 
    pad_center=True,
    resample_filter=PIL.Image.LANCZOS):
    
    if random_enhance:
        image = random_enhance_pil(image)
    
    width, height = image.size
    
    if height > width:
        new_height = size
        ratio = new_height / height
        new_width = int(width * ratio)
    
    elif width > height:
        new_width = size
        ratio = new_width / width
        new_height = int(height * ratio)
    
    else:
        new_width = size
        new_height = size
        
    image_resize = image.resize((new_width, new_height), resample=resample_filter)
    
    if pad:
        image_pad = Image.new("RGB", (size, size), (0, 0, 0))
      
        if pad_center:
            ulc = ((size - new_width) // 2, (size - new_height) // 2)
            image_pad.paste(image_resize, ulc)
        else:
            image_pad.paste(image_resize, (0, 0))

        return image_pad
    else:
        return image_resize

In [4]:
BRIGHTNESS_FACTOR_MIN = 0.25
BRIGHTNESS_FACTOR_MAX = 1.5

CONTRAST_FACTOR_MIN = 0.25
CONTRAST_FACTOR_MAX = 2

SHARPENESS_FACTOR_MIN = 0
SHARPENESS_FACTOR_MAX = 4

SATURATION_FACTOR_MIN = 0
SATURATION_FACTOR_MAX = 2

def random_enhance_pil(image, brightness=True, contrast=True, saturation=True, sharpness=True):
    new_image = image
    
    if brightness:
        enhancer = ImageEnhance.Brightness(new_image)
        new_image = enhancer.enhance(random.uniform(BRIGHTNESS_FACTOR_MIN, BRIGHTNESS_FACTOR_MAX))
    
    if contrast:
        enhancer = ImageEnhance.Contrast(new_image)
        new_image = enhancer.enhance(random.uniform(CONTRAST_FACTOR_MIN, CONTRAST_FACTOR_MAX))
    
    if saturation:
        enhancer = ImageEnhance.Color(new_image)
        new_image = enhancer.enhance(random.uniform(SATURATION_FACTOR_MIN, SATURATION_FACTOR_MAX))
    
    if sharpness:
        enhancer = ImageEnhance.Sharpness(new_image)
        new_image = enhancer.enhance(random.uniform(SHARPENESS_FACTOR_MIN, SHARPENESS_FACTOR_MAX))
    
    return new_image

In [5]:
def enhance_sharpness(image, factor):
    enhancer = ImageEnhance.Sharpness(image)
    new_image = enhancer.enhance(factor)
    
    return new_image

def enhance_contrast(image, factor):
    enhancer = ImageEnhance.Contrast(image)
    new_image = enhancer.enhance(factor)
    
    return new_image

In [6]:
def image_to_nparray_pil(image, shape):
    return np.array(image.getdata()).reshape(shape) 

In [7]:
def image_to_float_pil(image):
    return image.astype(float) / 255.0

In [8]:
def random_enhance_tf(image, brightness=True, contrast=True, saturation=True):
    new_image = image
    
    if brightness:
        seed = datetime.datetime.now().second
        new_image = tf.image.random_brightness(new_image, 5, seed=seed)
    
    if contrast:
        seed = datetime.datetime.now().second
        new_image = tf.image.random_contrast(new_image, 0.1, 1.5, seed=seed)
    
    if saturation:
        seed = datetime.datetime.now().second
        new_image = tf.image.random_saturation(new_image, 0., 1.5, seed=seed)
    
    return new_image

def random_flip_tf(image, vertical=True, horizontal=True):
    new_image = image
    
    if vertical:
        seed = datetime.datetime.now().second
        new_image = tf.image.random_flip_up_down(new_image, seed=seed)
    
    if horizontal:
        seed = datetime.datetime.now().second
        new_image = tf.image.random_flip_left_right(new_image, seed=seed)
    
    return new_image

In [9]:
# reference from kaggle competition kernel

def dice_coef(y_true, y_pred):
    y_pred_f = tf.contrib.keras.backend.flatten(y_pred)
    y_true_f = tf.contrib.keras.backend.flatten(y_true)
    
    intersection = tf.contrib.keras.backend.sum(y_true_f * y_pred_f)
    
    smooth = 0.0
    
    return (2.0 * intersection + smooth) / (tf.contrib.keras.backend.sum(y_true_f) + 
                                            tf.contrib.keras.backend.sum(y_pred_f) + smooth)

def dice_coef_batch(y_true_batch, y_pred_batch):
    y_pred_f = tf.contrib.layers.flatten(y_pred_batch)
    y_true_f = tf.contrib.layers.flatten(y_true_batch)
    
    intersection = tf.contrib.keras.backend.sum(tf.multiply(y_true_f, y_pred_f), axis=1)
    
    return (2.0 * intersection) / (tf.contrib.keras.backend.sum(y_true_f, axis=1) + 
                                            tf.contrib.keras.backend.sum(y_pred_f, axis=1))

def bce_dice_loss(y_true, y_pred):
    return 0.5 * tf.contrib.keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [10]:
def mse_error(y_true_batch, y_pred_batch):
    y_pred_f = tf.contrib.layers.flatten(y_pred_batch)
    y_true_f = tf.contrib.layers.flatten(y_true_batch)
    
    difference = tf.subtract(y_true_f, y_pred_f)
    
    return tf.square(difference)

In [11]:
# Global Settings

IMAGE_SIZE = 224 # ImageNet Models Compatibility

WIDTH = IMAGE_SIZE
HEIGHT = IMAGE_SIZE

n_outputs = IMAGE_SIZE * IMAGE_SIZE

# parameters of convolution layers
n_filters = [175, 175, 175]
n_kernel_size = [7, 5, 3]
n_strides = [2, 2, 2]

# parameters of fully connected layers
keep_prob = 0.75
n_neurons = [2048]

# parameters of optimizer
learning_rate = 0.01

# prefix = "convnet_sigmoid_{}".format("01")
# prefix = "perceptron_predict_{}".format("01")

In [12]:
def conv_graph(x, is_training, prefix):
    layer = x
#     label = y

    layer = tf.image.convert_image_dtype(layer, tf.float32)

#     label = tf.image.convert_image_dtype(label, tf.float32) / 255.0

    index = 0
    for filters, kernel_size, stride in zip(n_filters, n_kernel_size, n_strides):
        layer = conv2d(
            layer, 
            filters, 
            kernel_size, 
            stride, 
            padding="SAME",
            # use elu activation function
            activation_fn=tf.nn.elu,
            # use batch normalization
            normalizer_fn=batch_norm, 
            normalizer_params={
                "is_training": is_training,
                "decay": 0.99,
                "updates_collections": None,
            },
            # use he initializer
            weights_initializer=variance_scaling_initializer(), 
            scope="{}_conv-{}".format(prefix, index),
        )

        index += 1

    layer = tf.nn.avg_pool(
        layer, 
        ksize=[1, 4, 4, 1], 
        strides=[1, 2, 2, 1], 
        padding="SAME", 
        name="{}_avg-pool".format(prefix),
    )

    layer = tf.contrib.layers.flatten(layer)

    layer = dropout(
        layer,
        keep_prob,
        is_training=is_training,
        scope="{}_drop-to-fc".format(prefix),
    )

    index = 0
    for neurons in n_neurons:
        layer = fully_connected(
            layer, 
            neurons,
            # use elu activation function
            activation_fn=tf.nn.elu, 
            # use batch normalization
            normalizer_fn=batch_norm, 
            normalizer_params={
                "is_training": is_training,
                "decay": 0.99,
                "updates_collections": None,
            },
            # use he initializer
            weights_initializer=variance_scaling_initializer(), 
            scope="{}_fc-{}".format(prefix, index),
        )

        layer = dropout(
            layer,
            keep_prob,
            is_training=is_training,
            scope="{}_drop-{}".format(prefix, index),
        )

        index += 1

    logits = fully_connected(
        layer,
        n_outputs,
        # use no activation function
        activation_fn=None,
        # use batch normalization
        normalizer_fn=batch_norm, 
        normalizer_params={
            "is_training": True,
            "decay": 0.99,
            "updates_collections": None,
        },
        # use he initializer
        weights_initializer=variance_scaling_initializer(), 
        scope="{}_logits".format(prefix),
    )

    outputs = tf.nn.sigmoid(logits, name="{}_outputs".format(prefix))
    outputs_image = tf.reshape(outputs, [-1, IMAGE_SIZE, IMAGE_SIZE, 1])
    outputs_summary = tf.summary.image("outputs", outputs_image)
    
#     predict = fully_connected(
#         outputs,
#         n_outputs,
#         # use no activation function
#         activation_fn=None,
#         # use he initializer
#         weights_initializer=variance_scaling_initializer(), 
#         scope="{}_predict".format(prefix),
#     )

    predict = tf.rint(outputs, name="{}_predict".format(prefix))
    predict_image = tf.reshape(predict, [-1, IMAGE_SIZE, IMAGE_SIZE, 1])
    predict_summary = tf.summary.image("predict", predict_image)
    
    return outputs, outputs_summary, predict_summary

In [13]:
def training_graph(predict, label, prefix):
    
    label = tf.image.convert_image_dtype(label, tf.float32) / 255.0
    
    label_summary = tf.summary.image("label", label)

    loss = tf.reduce_mean(mse_error(label, predict))

    loss_summary = tf.summary.scalar("loss", loss)

    # use AdamOptimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

    # minimizing the loss in training op
    training_op = optimizer.minimize(loss)
    
    return training_op, label_summary, loss_summary

In [ ]:
def init_graph(prefix):
    # initializer for all variables
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()

    LOG_DIR = "logs/{}/".format(prefix)

    summary_writer = tf.summary.FileWriter(LOG_DIR, tf.get_default_graph())
    
    return init, saver, summary_writer

In [ ]:
for i in range(16):
    INDEX = "{:02d}".format(i+1)
    prefix = "convnet_sigmoid_{}".format(INDEX)
    # prefix = "perceptron_predict_{}".format(INDEX)
    
    tf.reset_default_graph()
    
    x = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 3])
    y = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 1])

    is_training = tf.placeholder(tf.bool, shape=())
    
    outputs, outputs_summary, predict_summary = conv_graph(x, is_training, prefix)
    training_op, label_summary, loss_summary = training_graph(outputs, y, prefix)
    init, saver, summary_writer = init_graph(prefix)

    n_epochs = 150
    batch_size = 10

    # runing the graph
    with tf.Session() as sess:

        # we need to initialize all variables first
        init.run()

    #     saver.restore(sess, "./my-convnet.ckpt")

        for epoch in range(n_epochs):

            TRAINING_DATASET = []

            for i in os.listdir(TRAIN_IMAGES):
                if os.path.splitext(i)[0].split("_")[1] == INDEX:
                    TRAINING_DATASET.append(os.path.join(TRAIN_IMAGES, i))

            TRAINING_DATASET = np.random.permutation(TRAINING_DATASET)

            LABEL_DATASET = [
                os.path.join(
                    TRAIN_MASKS, 
                    os.path.splitext(os.path.basename(i))[0]+"_mask.gif")
                for i in TRAINING_DATASET]

            for iteration in range(len(TRAINING_DATASET) // batch_size):
                # get mini bath of training data
                next_batch = get_next_batch(
                    batch_size=batch_size, 
                    iteration=iteration, 
                    dataset=TRAINING_DATASET)

                x_batch = []

                for file in next_batch:
                    image = Image.open(file)
    
                    image = resize_image_pil(
                        image, 
                        IMAGE_SIZE, 
                        random_enhance=True, 
                        pad=True, 
                        pad_center=False)
        
                    x_batch.append(image) 

                x_batch = [image_to_nparray_pil(image, (HEIGHT, WIDTH, 3)) for image in x_batch]

                next_batch = get_next_batch(
                    batch_size=batch_size, 
                    iteration=iteration, 
                    dataset=LABEL_DATASET)

                y_batch = []

                for file in next_batch:
                    image = Image.open(file)

                    image = image.convert("RGB")
                    
                    image = resize_image_pil(
                        image, 
                        IMAGE_SIZE, 
                        random_enhance=False, 
                        pad=True, 
                        pad_center=False)
                    
                    image = image.convert("L")

                    y_batch.append(image)

                y_batch = [image_to_nparray_pil(image, (HEIGHT, WIDTH, 1)) for image in y_batch]

                _, loss_log, outputs_log, predict_log, label_log = sess.run(
                    [
                        training_op,
                        loss_summary,
                        outputs_summary,
                        predict_summary, 
                        label_summary,
                    ], 
                    feed_dict={
                        x: x_batch, 
                        y: y_batch, 
                        is_training: True,
                    }
                )

                step = epoch * (len(TRAINING_DATASET) // batch_size) + iteration
                summary_writer.add_summary(loss_log, step)

                print(epoch, iteration)

                if iteration % 100 == 0:
                    saver.save(sess, "./{}/my-convnet.ckpt".format(prefix))

                if iteration % 10 == 0:  
                    summary_writer.add_summary(predict_log, step)
                    summary_writer.add_summary(label_log, step)
                    summary_writer.add_summary(outputs_log, step) 

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
4 30
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
5 25
5 26
5 27
5 28
5 29
5 30
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
6 24
6 25
6 26
6 27


48 11
48 12
48 13
48 14
48 15
48 16
48 17
48 18
48 19
48 20
48 21
48 22
48 23
48 24
48 25
48 26
48 27
48 28
48 29
48 30
49 0
49 1
49 2
49 3
49 4
49 5
49 6
49 7
49 8
49 9
49 10
49 11
49 12
49 13
49 14
49 15
49 16
49 17
49 18
49 19
49 20
49 21
49 22
49 23
49 24
49 25
49 26
49 27
49 28
49 29
49 30
50 0
50 1
50 2
50 3
50 4
50 5
50 6
50 7
50 8
50 9
50 10
50 11
50 12
50 13
50 14
50 15
50 16
50 17
50 18
50 19
50 20
50 21
50 22
50 23
50 24
50 25
50 26
50 27
50 28
50 29
50 30
51 0
51 1
51 2
51 3
51 4
51 5
51 6
51 7
51 8
51 9
51 10
51 11
51 12
51 13
51 14
51 15
51 16
51 17
51 18
51 19
51 20
51 21
51 22
51 23
51 24
51 25
51 26
51 27
51 28
51 29
51 30
52 0
52 1
52 2
52 3
52 4
52 5
52 6
52 7
52 8
52 9
52 10
52 11
52 12
52 13
52 14
52 15
52 16
52 17
52 18
52 19
52 20
52 21
52 22
52 23
52 24
52 25
52 26
52 27
52 28
52 29
52 30
53 0
53 1
53 2
53 3
53 4
53 5
53 6
53 7
53 8
53 9
53 10
53 11
53 12
53 13
53 14
53 15
53 16
53 17
53 18
53 19
53 20
53 21
53 22
53 23
53 24
53 25
53 26
53 27
53 28
53 29
53 30


94 28
94 29
94 30
95 0
95 1
95 2
95 3
95 4
95 5
95 6
95 7
95 8
95 9
95 10
95 11
95 12
95 13
95 14
95 15
95 16
95 17
95 18
95 19
95 20
95 21
95 22
95 23
95 24
95 25
95 26
95 27
95 28
95 29
95 30
96 0
96 1
96 2
96 3
96 4
96 5
96 6
96 7
96 8
96 9
96 10
96 11
96 12
96 13
96 14
96 15
96 16
96 17
96 18
96 19
96 20
96 21
96 22
96 23
96 24
96 25
96 26
96 27
96 28
96 29
96 30
97 0
97 1
97 2
97 3
97 4
97 5
97 6
97 7
97 8
97 9
97 10
97 11
97 12
97 13
97 14
97 15
97 16
97 17
97 18
97 19
97 20
97 21
97 22
97 23
97 24
97 25
97 26
97 27
97 28
97 29
97 30
98 0
98 1
98 2
98 3
98 4
98 5
98 6
98 7
98 8
98 9
98 10
98 11
98 12
98 13
98 14
98 15
98 16
98 17
98 18
98 19
98 20
98 21
98 22
98 23
98 24
98 25
98 26
98 27
98 28
98 29
98 30
99 0
99 1
99 2
99 3
99 4
99 5
99 6
99 7
99 8
99 9
99 10
99 11
99 12
99 13
99 14
99 15
99 16
99 17
99 18
99 19
99 20
99 21
99 22
99 23
99 24
99 25
99 26
99 27
99 28
99 29
99 30
100 0
100 1
100 2
100 3
100 4
100 5
100 6
100 7
100 8
100 9
100 10
100 11
100 12
100 13
100 14
100 15


135 9
135 10
135 11
135 12
135 13
135 14
135 15
135 16
135 17
135 18
135 19
135 20
135 21
135 22
135 23
135 24
135 25
135 26
135 27
135 28
135 29
135 30
136 0
136 1
136 2
136 3
136 4
136 5
136 6
136 7
136 8
136 9
136 10
136 11
136 12
136 13
136 14
136 15
136 16
136 17
136 18
136 19
136 20
136 21
136 22
136 23
136 24
136 25
136 26
136 27
136 28
136 29
136 30
137 0
137 1
137 2
137 3
137 4
137 5
137 6
137 7
137 8
137 9
137 10
137 11
137 12
137 13
137 14
137 15
137 16
137 17
137 18
137 19
137 20
137 21
137 22
137 23
137 24
137 25
137 26
137 27
137 28
137 29
137 30
138 0
138 1
138 2
138 3
138 4
138 5
138 6
138 7
138 8
138 9
138 10
138 11
138 12
138 13
138 14
138 15
138 16
138 17
138 18
138 19
138 20
138 21
138 22
138 23
138 24
138 25
138 26
138 27
138 28
138 29
138 30
139 0
139 1
139 2
139 3
139 4
139 5
139 6
139 7
139 8
139 9
139 10
139 11
139 12
139 13
139 14
139 15
139 16
139 17
139 18
139 19
139 20
139 21
139 22
139 23
139 24
139 25
139 26
139 27
139 28
139 29
139 30
140 0
140 1
140 2
1

31 0
31 1
31 2
31 3
31 4
31 5
31 6
31 7
31 8
31 9
31 10
31 11
31 12
31 13
31 14
31 15
31 16
31 17
31 18
31 19
31 20
31 21
31 22
31 23
31 24
31 25
31 26
31 27
31 28
31 29
31 30
32 0
32 1
32 2
32 3
32 4
32 5
32 6
32 7
32 8
32 9
32 10
32 11
32 12
32 13
32 14
32 15
32 16
32 17
32 18
32 19
32 20
32 21
32 22
32 23
32 24
32 25
32 26
32 27
32 28
32 29
32 30
33 0
33 1
33 2
33 3
33 4
33 5
33 6
33 7
33 8
33 9
33 10
33 11
33 12
33 13
33 14
33 15
33 16
33 17
33 18
33 19
33 20
33 21
33 22
33 23
33 24
33 25
33 26
33 27
33 28
33 29
33 30
34 0
34 1
34 2
34 3
34 4
34 5
34 6
34 7
34 8
34 9
34 10
34 11
34 12
34 13
34 14
34 15
34 16
34 17
34 18
34 19
34 20
34 21
34 22
34 23
34 24
34 25
34 26
34 27
34 28
34 29
34 30
35 0
35 1
35 2
35 3
35 4
35 5
35 6
35 7
35 8
35 9
35 10
35 11
35 12
35 13
35 14
35 15
35 16
35 17
35 18
35 19
35 20
35 21
35 22
35 23
35 24
35 25
35 26
35 27
35 28
35 29
35 30
36 0
36 1
36 2
36 3
36 4
36 5
36 6
36 7
36 8
36 9
36 10
36 11
36 12
36 13
36 14
36 15
36 16
36 17
36 18
36 19
36 20
36 2

...

55 19
55 20
55 21
55 22
55 23
55 24
55 25
55 26
55 27
55 28
55 29
55 30
56 0
56 1
56 2
56 3
56 4
56 5
56 6
56 7
56 8
56 9
56 10
56 11
56 12
56 13
56 14
56 15
56 16
56 17
56 18
56 19
56 20
56 21
56 22
56 23
56 24
56 25
56 26
56 27
56 28
56 29
56 30
57 0
57 1
57 2
57 3
57 4
57 5
57 6
57 7
57 8
57 9
57 10
57 11
57 12
57 13
57 14
57 15
57 16
57 17
57 18
57 19
57 20
57 21
57 22
57 23
57 24
57 25
57 26
57 27
57 28
57 29
57 30
58 0
58 1
58 2
58 3
58 4
58 5
58 6
58 7
58 8
58 9
58 10
58 11
58 12
58 13
58 14
58 15
58 16
58 17
58 18
58 19
58 20
58 21
58 22
58 23
58 24
58 25
58 26
58 27
58 28
58 29
58 30
59 0
59 1
59 2
59 3
59 4
59 5
59 6
59 7
59 8
59 9
59 10
59 11
59 12
59 13
59 14
59 15
59 16
59 17
59 18
59 19
59 20
59 21
59 22
59 23
59 24
59 25
59 26
59 27
59 28
59 29
59 30
60 0
60 1
60 2
60 3
60 4
60 5
60 6
60 7
60 8
60 9
60 10
60 11
60 12
60 13
60 14
60 15
60 16
60 17
60 18
60 19
60 20
60 21
60 22
60 23
60 24
60 25
60 26
60 27
60 28
60 29
60 30
61 0
61 1
61 2
61 3
61 4
61 5
61 6
61 7
61 8
61 

101 26
101 27
101 28
101 29
101 30
102 0
102 1
102 2
102 3
102 4
102 5
102 6
102 7
102 8
102 9
102 10
102 11
102 12
102 13
102 14
102 15
102 16
102 17
102 18
102 19
102 20
102 21
102 22
102 23
102 24
102 25
102 26
102 27
102 28
102 29
102 30
103 0
103 1
103 2
103 3
103 4
103 5
103 6
103 7
103 8
103 9
103 10
103 11
103 12
103 13
103 14
103 15
103 16
103 17
103 18
103 19
103 20
103 21
103 22
103 23
103 24
103 25
103 26
103 27
103 28
103 29
103 30
104 0
104 1
104 2
104 3
104 4
104 5
104 6
104 7
104 8
104 9
104 10
104 11
104 12
104 13
104 14
104 15
104 16
104 17
104 18
104 19
104 20
104 21
104 22
104 23
104 24
104 25
104 26
104 27
104 28
104 29
104 30
105 0
105 1
105 2
105 3
105 4
105 5
105 6
105 7
105 8
105 9
105 10
105 11
105 12
105 13
105 14
105 15
105 16
105 17
105 18
105 19
105 20
105 21
105 22
105 23
105 24
105 25
105 26
105 27
105 28
105 29
105 30
106 0
106 1
106 2
106 3
106 4
106 5
106 6
106 7
106 8
106 9
106 10
106 11
106 12
106 13
106 14
106 15
106 16
106 17
106 18
106 19
106 20


141 14
141 15
141 16
141 17
141 18
141 19
141 20
141 21
141 22
141 23
141 24
141 25
141 26
141 27
141 28
141 29
141 30
142 0
142 1
142 2
142 3
142 4
142 5
142 6
142 7
142 8
142 9
142 10
142 11
142 12
142 13
142 14
142 15
142 16
142 17
142 18
142 19
142 20
142 21
142 22
142 23
142 24
142 25
142 26
142 27
142 28
142 29
142 30
143 0
143 1
143 2
143 3
143 4
143 5
143 6
143 7
143 8
143 9
143 10
143 11
143 12
143 13
143 14
143 15
143 16
143 17
143 18
143 19
143 20
143 21
143 22
143 23
143 24
143 25
143 26
143 27
143 28
143 29
143 30
144 0
144 1
144 2
144 3
144 4
144 5
144 6
144 7
144 8
144 9
144 10
144 11
144 12
144 13
144 14
144 15
144 16
144 17
144 18
144 19
144 20
144 21
144 22
144 23
144 24
144 25
144 26
144 27
144 28
144 29
144 30
145 0
145 1
145 2
145 3
145 4
145 5
145 6
145 7
145 8
145 9
145 10
145 11
145 12
145 13
145 14
145 15
145 16
145 17
145 18
145 19
145 20
145 21
145 22
145 23
145 24
145 25
145 26
145 27
145 28
145 29
145 30
146 0
146 1
146 2
146 3
146 4
146 5
146 6
146 7
146 8

38 8
38 9
38 10
38 11
38 12
38 13
38 14
38 15
38 16
38 17
38 18
38 19
38 20
38 21
38 22
38 23
38 24
38 25
38 26
38 27
38 28
38 29
38 30
39 0
39 1
39 2
39 3
39 4
39 5
39 6
39 7
39 8
39 9
39 10
39 11
39 12
39 13
39 14
39 15
39 16
39 17
39 18
39 19
39 20
39 21
39 22
39 23
39 24
39 25
39 26
39 27
39 28
39 29
39 30
40 0
40 1
40 2
40 3
40 4
40 5
40 6
40 7
40 8
40 9
40 10
40 11
40 12
40 13
40 14
40 15
40 16
40 17
40 18
40 19
40 20
40 21
40 22
40 23
40 24
40 25
40 26
40 27
40 28
40 29
40 30
41 0
41 1
41 2
41 3
41 4
41 5
41 6
41 7
41 8
41 9
41 10
41 11
41 12
41 13
41 14
41 15
41 16
41 17
41 18
41 19
41 20
41 21
41 22
41 23
41 24
41 25
41 26
41 27
41 28
41 29
41 30
42 0
42 1
42 2
42 3
42 4
42 5
42 6
42 7
42 8
42 9
42 10
42 11
42 12
42 13
42 14
42 15
42 16
42 17
42 18
42 19
42 20
42 21
42 22
42 23
42 24
42 25
42 26
42 27
42 28
42 29
42 30
43 0
43 1
43 2
43 3
43 4
43 5
43 6
43 7
43 8
43 9
43 10
43 11
43 12
43 13
43 14
43 15
43 16
43 17
43 18
43 19
43 20
43 21
43 22
43 23
43 24
43 25
43 26
43 27
43

84 25
84 26
84 27
84 28
84 29
84 30
85 0
85 1
85 2
85 3
85 4
85 5
85 6
85 7
85 8
85 9
85 10
85 11
85 12
85 13
85 14
85 15
85 16
85 17
85 18
85 19
85 20
85 21
85 22
85 23
85 24
85 25
85 26
85 27
85 28
85 29
85 30
86 0
86 1
86 2
86 3
86 4
86 5
86 6
86 7
86 8
86 9
86 10
86 11
86 12
86 13
86 14
86 15
86 16
86 17
86 18
86 19
86 20
86 21
86 22
86 23
86 24
86 25
86 26
86 27
86 28
86 29
86 30
87 0
87 1
87 2
87 3
87 4
87 5
87 6
87 7
87 8
87 9
87 10
87 11
87 12
87 13
87 14
87 15
87 16
87 17
87 18
87 19
87 20
87 21
87 22
87 23
87 24
87 25
87 26
87 27
87 28
87 29
87 30
88 0
88 1
88 2
88 3
88 4
88 5
88 6
88 7
88 8
88 9
88 10
